# Install dependencies of this project

In [ ]:
!pip install -r requirements.txt

# Setup Azure ML

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

ml_client = MLClient(
    credential=credential,
    subscription_id="d128f140-94e6-4175-87a7-954b9d27db16",  # this will look like xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
    resource_group_name="jietong-test",
    workspace_name="jietong-test-4"
)

# Get familiar with the primary interface - PFClient

In [ ]:
import promptflow.azure as azure

pf = azure.PFClient(ml_client)

# Create connections
Connection in prompt flow is for managing settings of your application behaviors incl. how to talk to different services (Azure OpenAI for example).
In many applications, configuration files or environment variables are used for this purpose. Chat_with_pdf also uses environment variables, to make it work with prompt flow and without changing how environment variables are used, we populate everything in the CustomConnection into environment variables.
```python
def setup_env(conn: CustomConnection):
    if not conn:
        return
    for key in conn:
        os.environ[key] = conn[key]
```

chat_with_pdf requires following env vars (thus for the custom connection named "my_custom_connection"):
```
OPENAI_API_BASE=<AOAI_ENDPOINT>
OPENAI_API_VERSION=2023-03-15-preview
OPENAI_API_KEY=<AOAI_API_KEY>
EMBEDDING_MODEL_DEPLOYMENT_NAME=text-embedding-ada-002
CHAT_MODEL_DEPLOYMENT_NAME=gpt-35-turbo
PROMPT_TOKEN_LIMIT=3000
MAX_COMPLETION_TOKENS=256
```

from promptflow.entities import CustomConnection

if len([c for c in client.connections.list() if c.name == "my_custom_connection"]) == 0:
    # Create the custom connection that is required by chat_with_pdf_tool
    print("Creating custom connection: my_custom_connection")
    conn = CustomConnection(
        name = "my_custom_connection",
        configs= {
            "OPENAI_API_VERSION": "2023-03-15-preview",
            "EMBEDDING_MODEL_DEPLOYMENT_NAME": "text-embedding-ada-002",
            "CHAT_MODEL_DEPLOYMENT_NAME": "gpt-35-turbo",
            "PROMPT_TOKEN_LIMIT": "3000",
            "MAX_COMPLETION_TOKENS": "256"
        },
        secrets= {
            "OPENAI_API_BASE": "AOAI_ENDPOINT", # replace this
            "OPENAI_API_KEY": "AOAI_API_KEY", # replace this
        })
    client.connections.create_or_update(conn)
    print("Custom connection: my_custom_connection created.")
else:
    print("Custom connection: my_custom_connection found.")

# Run a flow with settings in custom connection (context size 3K)

In [ ]:
flow_path = "."
data_path = "./test_data/bert-paper-qna.jsonl"

run_3k_context = pf.run(
    flow=flow_path, 
    data=data_path, 
    connections={"setup_env": {"conn": "chat_with_pdf_custom_connection"}},
    runtime="chat_with_pdf_runtime",
    display_name="chat_with_pdf_3k_context",
    tags={"chat_with_pdf":"", "2nd_round":""},
    stream=True)
pf.stream(run_3k_context)

print(run_3k_context)

In [ ]:
detail = pf.get_details(run_3k_context)

detail

# Run a flow with settings in custom connection (context size 2K)

In [ ]:
flow_path = "."
data_path = "./test_data/bert-paper-qna.jsonl"

run_2k_context = pf.run(
    flow=flow_path, 
    data=data_path, 
    connections={"setup_env": {"conn": "chat_with_pdf_custom_connection_smaller_context"}},
    runtime="chat_with_pdf_runtime",
    display_name="chat_with_pdf_2k_context",
    tags={"chat_with_pdf":"", "2nd_round":""},
    stream=True)
pf.stream(run_2k_context)

print(run_2k_context)

In [ ]:
detail = pf.get_details(run_2k_context)

detail